# Execution num1:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 3
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 5,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None


Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:17662
Please start proper ns-3 simulation script using ./waf --run "..."
6.723999|6.589966|6.589966|7.148438|6.511780|0|0|0|0|0|5|-nan
Observation space shape: (1, 300)
Action space shape: (1, 1)


In [4]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4

agent_count = 5 # Number of agents to train
agents = []    # List to store the agents

for i in range(agent_count):
    config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
    agent = Agent(QNetworkTf, history_length, action_size=7, stationNum=i, config=config)
    agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)
    agents.append(agent)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agents, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agents,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
#                        parameters=hyperparams)
# agent.save()


# %%

local0


target0
Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


local1


target1
Action space: 7


local2
target2
Action space: 7


...

0
6.723999|6.589966|6.589966|7.148438|6.511780|0|0|0|0|0|5|-nan


  5%|▌         | 333/6300 [00:47<15:42,  6.33it/s, curr_speed=35.39 Mbps, mb_sent=11.33 Mb]

------- STARTED TRAINING -------
------- STARTED TRAINING -------
------- STARTED TRAINING -------
------- STARTED TRAINING -------
------- STARTED TRAINING -------


 29%|██▉       | 1829/6300 [05:46<14:07,  5.27it/s, curr_speed=34.33 Mbps, mb_sent=545.87 Mb]


KeyboardInterrupt: 

In [1]:
rx_vector = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.07, 0.08, 0.09, 1]

In [2]:
tx_vector = [0.11, 0.22, 0.33, 0.44, 0.55, 0.66, 1.0, 0.18, 0.019, 0.2]

In [3]:
import numpy as np

MAX_WIFI = 50
last_rx = np.zeros(MAX_WIFI, dtype=np.uint64)
last_tx = np.zeros(MAX_WIFI, dtype=np.uint64)

received_total = 0.0
sent_total = 0.0
ratio_total = None
errs = 0.0

In [4]:
for i in range(10):
    received = rx_vector[i] - last_rx[i];        
         
    sent = tx_vector[i] - last_tx[i];      
         
    # soma todos os pacotes recebidos de todas as stations
    received_total = received_total + received;
    print("rx_T: ", received_total)
         
    # soma todos os pacotes transmitidos de todas as stations
    sent_total = sent_total + sent;
    print("tx_T: ", sent_total)
         
    errs = sent_total - received_total;    
         
    if(errs < 0):
        ratio_total = 0;
        print(ratio_total)
    else:
        ratio_total = errs / sent_total;
    
    print("ratio_total: ", ratio_total)
           
    
    last_rx[i] = rx_vector[i];
    last_tx[i] = tx_vector[i];

rx_T:  0.1
tx_T:  0.11
ratio_total:  0.09090909090909087
rx_T:  0.30000000000000004
tx_T:  0.33
ratio_total:  0.09090909090909081
rx_T:  0.6000000000000001
tx_T:  0.66
ratio_total:  0.09090909090909081
rx_T:  1.0
tx_T:  1.1
ratio_total:  0.09090909090909098
rx_T:  1.5
tx_T:  1.6500000000000001
ratio_total:  0.09090909090909098
rx_T:  2.1
tx_T:  2.31
ratio_total:  0.0909090909090909
rx_T:  2.17
tx_T:  3.31
ratio_total:  0.34441087613293053
rx_T:  2.25
tx_T:  3.49
ratio_total:  0.35530085959885394
rx_T:  2.34
tx_T:  3.5090000000000003
ratio_total:  0.33314334568253073
rx_T:  3.34
tx_T:  3.7090000000000005
ratio_total:  0.09948773254246444


In [5]:
from collections import deque
history = [deque() for _ in range(MAX_WIFI)]

print(received_total)
print(sent_total)

for i in range(10):
    history[i].append(ratio_total);
print(history)

3.34
3.7090000000000005
[deque([0.09948773254246444]), deque([0.09948773254246444]), deque([0.09948773254246444]), deque([0.09948773254246444]), deque([0.09948773254246444]), deque([0.09948773254246444]), deque([0.09948773254246444]), deque([0.09948773254246444]), deque([0.09948773254246444]), deque([0.09948773254246444]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([]), deque([])]
